In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [3]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [4]:
text = text[300000:800000]

In [5]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [6]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []

In [7]:
# From here next 4 cell are not needed 'afte training' the model and saving it

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i+SEQ_LENGTH])
    next_char.append(text[i+SEQ_LENGTH])


In [8]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1


In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))


c:\Users\ARSALAN\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.01))

model.fit(x,y, batch_size=256, epochs=4)

# After training the model, the model is saved in next step so we don't have to train it again and again
# and we will be using that file(model)

Epoch 1/4
 18/651 ━━━━━━━━━━━━━━━━━━━━ 1:59 188ms/step - loss: 3.3950

In [11]:
model.save('textgenerator.keras') # Change the file name to include .keras extension

In [12]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [15]:
print(generate_text(300, 0.2))
print(generate_text(300, 0.4))
print(generate_text(300, 0.5))
print(generate_text(300, 0.6))
print(generate_text(300, 0.7))
print(generate_text(300, 0.8))

 nay, the valley,
the pretty dimples of the will the world.

romeo:
and i should the world the streather and stready,
the brother and the will and the some his stand
the cread and streather and the love and the soul,
and the soul should the world the son and streath,
the soul should the world the warter then shall the world.

king richard
ose thy lips.

king henry vi:
i prithee, then the lord: the sacress of seeks
in the king a more and the crawner from the king.

king richard ii:
i have the parton the warward but are surpess.

cloffory:
i should my son the were the warter of the world.

cloffory:
i should the callace of the world should her so sull.
but i say the down tha
and, that was wont to conquer others,
hath the shall be the renting to merch.

york:
i shall be are my earth the may your lord.

romeo:
the forth the heavens consented the creath.

gloucester:
and the mistring his gards un all and here shall will.

mercutio:
no lord, and but the ponton that the warth,
there the rest 